## Simple Multi Layer Perceptron using Tensorflow

### Read the data

In [60]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
# heart desease
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

--2019-03-24 15:15:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18461 (18K) [text/plain]
Saving to: ‘processed.cleveland.data.1’

processed.cleveland 100%[===================>]  18.03K   103KB/s    in 0.2s    

2019-03-24 15:15:30 (103 KB/s) - ‘processed.cleveland.data.1’ saved [18461/18461]



In [61]:
! head ./processed.cleveland.data
#header=["age","sex","chestPainType","RestingBloodPressure","cholestoral","bloodSugar","restingElectroCardio","maxHeartRate"
#,"excerciseInducedAngine","oldpeak","slope","numColoredVessels","thal","target"]

63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [75]:
data=np.loadtxt("./processed.cleveland.data",delimiter=",",dtype=str)
data=np.where(data=='?',-1,data) # convert unknowns
X,y=data[:,:13],data[:,13]
X=np.ndarray.astype(X,np.float32)
y=np.ndarray.astype(y,np.int)
X,y

(array([[63.,  1.,  1., ...,  3.,  0.,  6.],
        [67.,  1.,  4., ...,  2.,  3.,  3.],
        [67.,  1.,  4., ...,  2.,  2.,  7.],
        ...,
        [57.,  1.,  4., ...,  2.,  1.,  7.],
        [57.,  0.,  2., ...,  2.,  1.,  3.],
        [38.,  1.,  3., ...,  1., -1.,  3.]], dtype=float32),
 array([0, 2, 1, 0, 0, 0, 3, 0, 2, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 3, 4, 0, 0, 0, 0, 3, 0, 2, 1, 0, 0, 0, 3, 1, 3, 0, 4, 0, 0, 0,
        1, 4, 0, 4, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0, 0, 2, 0, 1, 0, 2, 2,
        1, 0, 2, 1, 0, 3, 1, 1, 1, 0, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0,
        0, 0, 0, 3, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 3, 0, 2, 1, 2, 3,
        1, 1, 0, 2, 2, 0, 0, 0, 3, 2, 3, 4, 0, 3, 1, 0, 3, 3, 0, 0, 0, 0,
        0, 0, 0, 0, 4, 3, 1, 0, 0, 1, 0, 1, 0, 1, 4, 0, 0, 0, 0, 0, 0, 4,
        3, 1, 1, 1, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 4, 1,
        0, 1, 0, 0, 3, 2, 0, 0, 1, 0, 0, 2, 1, 2, 0, 3, 1, 2, 0, 3, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 3, 3,

In [76]:
X.shape, y.shape, set(y)

((303, 13), (303,), {0, 1, 2, 3, 4})

### Model

In [17]:
import tensorflow as tf
import numpy as np

In [68]:
feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(X) # infer which columns to use
feature_columns

[_RealValuedColumn(column_name='', dimension=13, default_value=None, dtype=tf.float32, normalizer=None)]

In [141]:
# hard core overfit the data
classifier=tf.contrib.learn.DNNClassifier(hidden_units=[300,300],n_classes=5,feature_columns=feature_columns)
classifier=tf.contrib.learn.SKCompat(classifier) # class for scikit lear compatibility
classifier.fit(X,y, batch_size=303, steps=4000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fade0254ef0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpqt76j4hh'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpqt76j4hh/mo

SKCompat()

In [137]:
classifier.predict(X)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp51l92bpx/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([[10.477915 ,  6.8609123,  6.4105887,  5.65363  , -1.8536805],
        [ 3.8873146, 15.566864 , 19.367874 ,  4.425101 ,  7.3757496],
        [ 4.321612 ,  7.296042 ,  6.4735456,  6.42986  ,  1.7329963],
        ...,
        [ 4.8231063,  3.1073997,  5.4713407, 14.815974 ,  7.2832756],
        [11.867576 , 12.018106 ,  4.362502 ,  6.1998825, -5.858029 ],
        [26.747648 ,  6.0012712, 10.266795 ,  6.683217 , -3.079881 ]],
       dtype=float32),
 'probabilities': array([[9.5054960e-01, 2.5534688e-02, 1.6276369e-02, 7.6350947e-03,
         4.1920935e-06],
        [1.8504475e-07, 2.1859517e-02, 9.7813386e-01, 3.1683567e-07,
         6.0573793e-06],
        [2.6674587e-02, 5.2224737e-01, 2.2944078e-01, 2.1963334e-01,
         2.0039012e-03],
        ...,
        [4.5693945e-05, 8.2174502e-06, 8.7374290e-05, 9.9932384e-01,
         5.3492980e-04],
        [4.6158314e-01, 5.3656781e-01, 2.5400243e-04, 1.5951520e-03,
         9.2494918e-09],
        [9.9999988e-01, 9.7715469

In [138]:
from sklearn.metrics import accuracy_score
accuracy_score(classifier.predict(X)["classes"],y)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp51l92bpx/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.966996699669967

In [139]:
from sklearn.ensemble import RandomForestClassifier

rfr=RandomForestClassifier()
rfr.fit(X,y)


/home/lweichbrodt/.conda/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [140]:
accuracy_score(rfr.predict(X),y)

0.9801980198019802